In [1]:
import sys
import math

class FastAreader:
    def __init__(self, fname=''):
        """contructor: saves attribute fname"""
        self.fname = fname
        self.fileH = None
        
    def doOpen(self):
        if self.fname == '':
            return sys.stdin
        else:
            return open(self.fname)
    
    def readFasta(self):
        header = ''
        sequence = ''
        with self.doOpen() as self.fileH:
            header = ''
            sequence = ''
            # skip to first fasta header
            line = self.fileH.readline()
            while not line.startswith('>'):
                line = self.fileH.readline()
            header = line[1:].rstrip()
            
            for line in self.fileH:
                if line.startswith('>'):
                    yield header, sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else:
                    sequence += ''.join(line.rstrip().split()).upper()
        yield header, sequence

In [2]:
fr = FastAreader("data/Ecoli-UMN026 (2).txt")
dna = list(fr.readFasta())[0][1]

In [3]:
class CommandLine():
    '''
    Handle the command line, usage and help requests.
    CommandLine uses argparse, now standard in 2.7 and beyond.
    it implements a standard command line argument parser with various argument options,
    a standard usage and help, and an error termination mechanism do-usage_and_die.
    attributes:
    all arguments received from the commandline using .add_argument will be
    avalable within the .args attribute of object instantiated from CommandLine.
    For example, if myCommandLine is an object of the class, and requiredbool was
    set as an option using add_argument, then myCommandLine.args.requiredbool will
    name that option.
    '''
    
    def __init__(self, inOpts=None):
        '''
        CommandLine constructor.
        Implements a parser to interpret the command line argv string using argparse.
        '''
        import argparse
        self.parser = argparse.ArgumentParser(
        description='Program prolog - a brief description of what this thing does',
        epilog='Program epilog - some other stuff you feel compelled to say',
        add_help=True, # default is True
        prefix_chars='-',
        usage='%(prog)s [options] -option1[default] <input >output'
        )
        
        self.parser.add_argument('-l', '--minMotif', nargs='?', type=int, default=1, action='store',
        help='min kMer size ')
        
        self.parser.add_argument('-m', '--maxMotif', nargs='?', type=int, default=8, action='store',
        help='max kMer size ')
        
        self.parser.add_argument('-c', '--cutoff', nargs='?', type=float, default=.01, action='store',
        help='Zscore cutoff')
        
        self.parser.add_argument('-v', '--version', action='version', version='%(prog)s 0.1')
        
        if inOpts is None:
            self.args = self.parser.parse_args()
        else:
            self.args = self.parser.parse_args(inOpts)

In [4]:
class Genome:
    """ """
    def __init__(self, minK, maxK):
        """Construct the object to count and analyze kMer frequencies"""
        self.mink = minK
        self.maxk = maxK
        
    def reverse_complement(self, dna):
        basepairs = {"A":"T", "G":"C", "T":"A", "C":"G"}
        return "".join([basepairs[base] for base in dna])[::-1]
        
    def E(self, K, counts, N):
        ex = 1
        for i in range(len(K)):
            if i == 0:
                ex *= (counts[K[i]]/N)
            else:
                kmer = K[:i+1]
                ex *= (counts[kmer]/counts[kmer[:-1]])
        return ex
        
    def count(self, genome):
        counts = {}
        bases = {'A', 'C', 'G', 'T'}
        for i in range(1, self.maxk+1):
            for k in range(len(genome)-i+1):
                kmer = genome[k:k+i]
                if set(kmer).issubset(bases):
                    counts[kmer] = counts.get(kmer, 0) + 1
        return counts
    
    def z_score(self, s, mu, sd):
        try:
            return (s - mu)/sd
        except ZeroDivisionError:
            return 0

In [ ]:
def main (inFile=None, options = None):
    ''' Setup necessary objects, read data and print the final report.'''
    cl = CommandLine(options) # setup the command line
    sourceReader = FastAreader(inFile) # setup the Fasta reader Object
    thisGenome = Genome(cl.args.minMotif, cl.args.maxMotif) # setup a Genome object
    ###
    # Give your Genome some data and tell it to do stuff
    for Id, genome in sourceReader.readFasta():
        counts = thisGenome.count(genome)
        for kmer, s in counts.items():
            if len(kmer) >= cl.args.minMotif:
                rev_kmer = thisGenome.reverse_complement(kmer)
                if (kmer == rev_kmer) or (rev_kmer not in counts.keys()):
                    total_count = counts[kmer]
                else:
                    total_count = counts[kmer] + counts[rev_kmer]
                    
                n = len(genome) - 8
                if (kmer == rev_kmer) or (rev_kmer not in counts.keys()):
                    ex = (thisGenome.E(kmer, counts, n) + thisGenome.E(kmer, counts, n)) * (2*n)
                else:
                    ex = thisGenome.E(kmer, counts, n) + thisGenome.E(rev_kmer, counts, n) * (2*n)
                p = ex/n
                mu = n*p
                sd = (mu * (1-p))**0.5
                zscore = thisGenome.z_score(s, mu, sd)
                #print(s, mu, sd, zscore)
                if zscore < cl.args.cutoff:
                    print(s, mu, sd, zscore)
    ###
    
if __name__ == "__main__":
    main(inFile="data/Ecoli-UMN026 (2).txt", options=["--minMotif=3", "--maxMotif=8", "--cutoff=-4."])